# Feature Generation
## Generalised for Tyrosine Phosphorylation 
#### Author: Nashira H. Ridgeway
#### Date Created: Tuesday, October 1st, 2024
#### Date Modified: Wednesday, October 16th , 2024
####
This code further generalises the manual generation of features for the PML pipeline.
Modified to accommodate for phosphorylation/kinase based datasets, this code enables the user to generate the MACCS key, ProtDCal, and one-hot encoding features required to run the hybrid method on peptide array data. 

Dr. Gianni Cesareni and Dr. Michele Tinti provided guidance regarding their dataset from the 2017 publication, "Both Intrinsic Substrate Preference and Network Context Contribute to Substrate Selection of Classical Tyrosine Phosphatases" which contained a dataset of 6,400 tyrosine-centric peptides representative of potential substrates within proteins tested with over 15 different phosphorylases. This extensive dataset will be applied to generate 15 different ML models to predict each corresponding phosphorylases' activity towards tyrosines across the proteome. 

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdmolfiles
import itertools
from collections import Counter

In [2]:
# Import file containing peptide sequences, protein information, and experimental values
#  for the different 15 enzymes analysed within the investigation
peptides = pd.read_csv('./PTP_Palma_et_al_2017.csv')

In [3]:
# Note -- there are two columns for PTP1B, each noted with a **
#  as there were two columns presented within the dataset with no discernible difference
#  other than a repeat in the experiment, these will be averaged to represent the PTP1B
#  activity across the 6,400 peptides monitored
PTP1B = list([col for col in peptides.columns if '**' in col])

mean = peptides[[PTP1B[0], PTP1B[1]]].mean(axis=1)
stdev = peptides[[PTP1B[0], PTP1B[1]]].std(axis=1)

In [4]:
# Remove dual columns
peptides = peptides.drop(columns=PTP1B)

In [5]:
# Replace with mean of both PTP1B columns
peptides['PTP_PTP1B_averaged'] = mean

In [6]:
# Import file containing protdcal values (needed for feature generation later on)
protdcal = pd.read_csv('../protdcal_features.csv', index_col=0)

In [7]:
# Not all peptides are of length 13 - some end prematurely or start late
peptides['Length'] = peptides['Interactor'].str.len()

# Show the peptides that aren't 13 AAs long 
peptides[peptides['Length'] != 13]

,Spot Index,Spot Flag,Interactor,Interactor Protein,Measure Flag,PTP_HD-PTP_11-07-2008.seam,PTP_LAR_12-10-2007.seam,PTP_LyP_12-10-2007.seam,PTP_MEG-1_12-10-2007.seam,PTP_MEG-2_12-10-2007.seam,...,PTP_PTPH1_12-10-2007.seam,PTP_rPTP-alpha_12-10-2007.seam,PTP_rPTP-beta_12-10-2007.seam,PTP_SAP-1_12-10-2007.seam,PTP_SHP-1_12-10-2007.seam,PTP_SHP-2_12-10-2007.seam,PTP_DEP-1_12-10-2007.seam.txt,PTP_TC-PTP_12-10-2007.seam,PTP_PTP1B_averaged,Length
96,3498,GOOD,YNSVVLYSTPPI,P17948 (FLT1) --> 1327-1338,GOOD,2.18000,1.4800,2.21000,1.5900,3.17000,...,2.16000,1.8300,2.73000,1.26000,0.4040,2.2100,3.86000,1.890000,1.12500,12
233,3506,GOOD,REFLDQYDAPL,O14543 (SOCS3) --> 215-225,GOOD,0.92800,0.2710,1.35000,1.8900,1.04000,...,2.04000,0.0730,0.66100,0.71800,2.6800,2.0300,2.02000,0.589000,0.75150,11
254,4324,GOOD,LYHKKSY,Q9HD26 (GOPC) --> 456-462,GOOD,-0.10000,0.5570,0.21300,-0.0200,0.09560,...,0.59100,1.2400,1.30000,1.41000,-0.0112,0.1800,1.81000,0.337000,0.57800,7
266,6157,GOOD,VQKSKEYFSKQK,Q06830 (PRDX1) --> 188-199,GOOD,0.28000,0.8740,1.08000,-0.3570,0.76500,...,0.82400,0.2130,0.41200,0.55200,0.0561,0.9960,1.72000,0.189000,0.34650,12
431,3373,GOOD,TQDSGFY,P60228 (EIF3S6) --> 439-445,GOOD,0.14700,0.9670,0.21300,-0.1330,-0.01840,...,0.17400,-0.0791,0.62100,0.55400,0.2350,0.1750,0.97800,0.012100,-0.02115,7
449,3522,GOOD,TRRTPDYFL,P62714 (PPP2CB) --> 301-309,GOOD,0.20700,2.2700,0.25300,0.4100,1.20000,...,0.25400,0.5660,1.56000,0.61000,0.9080,0.7430,0.95300,0.722000,0.51800,9
477,6161,GOOD,PYATSLYHS,Q8IV50-2 --> 116-124,GOOD,0.24700,0.1160,-0.34800,0.0267,0.26500,...,1.39000,0.2920,1.50000,2.94000,-1.3800,-0.1690,0.87100,1.200000,1.40850,9
518,4301,GOOD,RAVENQYSFY,O14492 (APS) --> 623-632,GOOD,0.29400,1.6400,0.76700,0.5870,0.51100,...,3.05000,1.7500,1.19000,0.41500,0.3030,1.0300,0.77300,1.290000,0.57900,10
533,3502,GOOD,GTPEGLYL,P62191 (PSMC1) --> 433-440,GOOD,0.00668,0.0155,0.90100,0.9610,0.20600,...,0.59800,-0.0548,0.53100,0.78200,-0.2800,0.7040,0.74500,0.921000,0.19900,8
578,4958,GOOD,DEEEDEYSGGLC,Q99747 (NAPG) --> 301-312,GOOD,0.41400,-0.0387,0.80600,0.5840,0.18000,...,0.02610,-0.3410,0.77800,0.14200,2.4900,1.5500,0.67100,1.120000,0.26850,12


In [8]:
# To address the length issue, we should first separate the "Interactor Protein" column
#  as it contains vital information about where our peptide is located within the full
#  length protein that can tell us how much is missing - we'll also require this
#  information to generate our Secondary ML scores (via MuSite Deep) and the Uniprot IDs
#  may be utilised to generate informative GO Annotations for future graphics

# Split up the information in the Interactor Protein column
temp = peptides['Interactor Protein'].str.split(' ', expand=True)
na_temp = temp[temp[1].isna()]
temp = temp.drop(na_temp.index)

# Create a column containing the Uniprot IDs
peptides['Uniprot_ID'] = temp[0]
peptides.loc[na_temp.index, 'Uniprot_ID'] = 'None_Listed'

# Create a column containing the conventional gene names
peptides['Gene_Name'] = temp[1]
peptides.loc[temp[temp[1].str.contains('>')].index, 'Gene_Name'] = 'None_Listed'
peptides.loc[na_temp.index, 'Gene_Name'] = 'None_Listed'

# Create a column containing the location of the peptide within the protein
peptides.loc[temp[temp[2].str.contains('>') == False].index, 'Peptide_Location'] = temp[temp[2].str.contains('>') == False][2]
peptides.loc[temp[temp[3].str.contains('>') == False].index, 'Peptide_Location'] = temp[temp[3].str.contains('>') == False][3]
peptides.loc[temp[temp[4].str.contains('>') == False].index, 'Peptide_Location'] = temp[temp[4].str.contains('>') == False][4]
peptides.loc[na_temp.index, 'Peptide_Location'] = 'None_Listed'

# Remove brackets from Gene_Name column
peptides.loc[peptides['Gene_Name'] != 'None_Listed', 'Gene_Name'] = peptides.loc[peptides['Gene_Name'] != 'None_Listed', 'Gene_Name'].str[1:-1]

In [9]:
# Read out peptide IDs to a file so we may retrieve the full length proteins from Uniprot
uids = peptides.loc[peptides['Uniprot_ID'] != 'None_Listed', 'Uniprot_ID'].drop_duplicates()
uids.to_csv('./Palma_et_al_2017_uniprot_IDs.csv', index=None)

In [10]:
# Retrieved full length sequences from uniprot (some matches not found/deleted)
fullseqs = pd.read_csv('./Palma_et_al_2017_uniprot_fullseqs.tsv', sep='\t')

In [11]:
fullseqs = fullseqs.drop_duplicates(subset='From')

In [12]:
fullseqs

,From,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence
0,Q10588,Q10588,BST1_HUMAN,ADP-ribosyl cyclase/cyclic ADP-ribose hydrolas...,BST1,Homo sapiens (Human),318.0,MAAQGCAASRLLQLLLQLLLLLLLLAAGGARARWRGEGTSAHLRDI...
1,P46019,P46019,KPB2_HUMAN,Phosphorylase b kinase regulatory subunit alph...,PHKA2 PHKLA PYK,Homo sapiens (Human),1235.0,MRSRSNSGVRLDGYARLVQQTILCYQNPVTGLLSASHEQKDAWVRD...
2,Q14980,Q14980,NUMA1_HUMAN,Nuclear mitotic apparatus protein 1 (Nuclear m...,NUMA1 NMP22 NUMA,Homo sapiens (Human),2115.0,MTLHATRGAALLSWVNSLHVADPVEAVLQLQDCSIFIKIIDRIHGT...
3,O60927,O60927,PP1RB_HUMAN,E3 ubiquitin-protein ligase PPP1R11 (EC 2.3.2....,PPP1R11 HCGV TCTE5,Homo sapiens (Human),126.0,MAEAGAGLSETVTETTVTVTTEPENRSLTIKLRKRKPEKKVEWTSD...
4,Q9ULT8,Q9ULT8,HECD1_HUMAN,E3 ubiquitin-protein ligase HECTD1 (EC 2.3.2.2...,HECTD1 KIAA1131,Homo sapiens (Human),2610.0,MADVDPDTLLEWLQMGQGDERDMQLIALEQLCMLLLMSDNVDRCFE...
...,...,...,...,...,...,...,...,...
3935,P29120,P29120,NEC1_HUMAN,Neuroendocrine convertase 1 (NEC 1) (EC 3.4.21...,PCSK1 NEC1,Homo sapiens (Human),753.0,MERRAWSLQCTAFVLFCAWCALNSAKAKRQFVNEWAAEIPGGPEAA...
3936,Q99777,Q99777,Q99777_HUMAN,Uncharacterized protein,NaN,Homo sapiens (Human),133.0,GAGVLHQHFSSGHIYVLMGLLPPPWTISFTVQTTLQPPGGLPAAPV...
3937,Q9H4A3,Q9H4A3,WNK1_HUMAN,Serine/threonine-protein kinase WNK1 (EC 2.7.1...,WNK1 HSN2 KDP KIAA0344 PRKWNK1,Homo sapiens (Human),2382.0,MSGGAAEKQSSTPGSLFLSPPAPAPKNGSSSDSSVGEKLGAAAADA...
3938,Q9UPC6,A8K7I4,CLCA1_HUMAN,Calcium-activated chloride channel regulator 1...,CLCA1 CACC1,Homo sapiens (Human),914.0,MGPFKSSVFILILHLLEGALSNSLIQLNNNGYEGIVVAIDPNVPED...


In [13]:
# Determine where the peptide sequence falls within our full sequence
loc = []
lgt = []

for r,h in peptides.iterrows():
    ui = h['Uniprot_ID']
    pep = h['Interactor']
    
    findin = fullseqs.loc[fullseqs['From'] == ui, 'Sequence']
    length = fullseqs.loc[fullseqs['From'] == ui, 'Length']
    
    if len(findin) < 1:
        loc.append('None_Available')
        lgt.append('None_Applicable')
    elif len(findin) == 1:
        try:
            loc.append(findin.str.index(pep).item())
            lgt.append(length.item())
        except:
            loc.append('Peptide_Not_Found')
            lgt.append('None_Applicable')

In [14]:
peptides['Peptide_Location_in_Fullseq'] = loc

In [15]:
peptides['Fullseq_Length'] = lgt

In [16]:
short_peps = peptides[peptides['Length'] != 13]

In [17]:
nonconforming = short_peps[short_peps['Fullseq_Length'].astype(str) == 'None_Applicable']
conforming = short_peps[short_peps['Fullseq_Length'].astype(str) != 'None_Applicable']

In [18]:
# Fix the conforming peptides with info gleamed from the full sequences
conforming['Tyr_Location'] = conforming['Peptide_Location_in_Fullseq'] + 7

/var/folders/p8/7dg76m153tbg1p6ln0gknfvw0000gn/T/ipykernel_10477/2914685964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conforming['Tyr_Location'] = conforming['Peptide_Location_in_Fullseq'] + 7


In [19]:
conforming['End_of_Peptide'] = conforming['Tyr_Location'] + 6

/var/folders/p8/7dg76m153tbg1p6ln0gknfvw0000gn/T/ipykernel_10477/772911962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conforming['End_of_Peptide'] = conforming['Tyr_Location'] + 6


In [20]:
# Issue with a conforming peptide - the peptide sequence is missing an amino acid
#  that is present within the full sequence. The singular amino acid likely shouldn't have
#  an effect, so it will be added in.

Q9BTX7_seq = fullseqs[fullseqs['From'] == 'Q9BTX7']['Sequence'].item()[62:(62+13)]
conforming.loc[conforming['Uniprot_ID'] == 'Q9BTX7', 'Interactor'] = Q9BTX7_seq

In [21]:
for h,r in conforming.iterrows():
    pep_start = r['Peptide_Location_in_Fullseq']
    pep_end = r['End_of_Peptide']
    prot_length = r['Fullseq_Length']
    pep_seq = r['Interactor']
    pep_id = r['Uniprot_ID']
    
    if len(pep_seq) != 13:

        if pep_end > prot_length:
            ala =  int(pep_end - prot_length)
            print('End of', pep_end, 'goes beyond', prot_length, 'by', ala)
            print(pep_seq)
            print((pep_seq + 'A'*ala))
            conforming.loc[h, 'Interactor'] = (pep_seq + 'A'*ala)
            
        elif pep_end < prot_length:
            ala = ((pep_start + 12) - pep_end)*-1
            print('Beginning of', pep_start, 'missing', ala, 'amino acids')
            print(pep_seq)
            print('A'*ala + pep_seq)
            conforming.loc[h, 'Interactor'] = ('A'*ala + pep_seq)
        else:
            print("\n\nANOTHER REASON?")
            print(pep_seq)
            print("\n\n")

End of 1339 goes beyond 1338.0 by 1
YNSVVLYSTPPI
YNSVVLYSTPPIA
End of 227 goes beyond 225.0 by 2
REFLDQYDAPL
REFLDQYDAPLAA
End of 468 goes beyond 462.0 by 6
LYHKKSY
LYHKKSYAAAAAA
End of 200 goes beyond 199.0 by 1
VQKSKEYFSKQK
VQKSKEYFSKQKA
End of 451 goes beyond 445.0 by 6
TQDSGFY
TQDSGFYAAAAAA
End of 313 goes beyond 309.0 by 4
TRRTPDYFL
TRRTPDYFLAAAA
End of 219 goes beyond 215.0 by 4
PYATSLYHS
PYATSLYHSAAAA
End of 635 goes beyond 632.0 by 3
RAVENQYSFY
RAVENQYSFYAAA
End of 445 goes beyond 440.0 by 5
GTPEGLYL
GTPEGLYLAAAAA
End of 313 goes beyond 312.0 by 1
DEEEDEYSGGLC
DEEEDEYSGGLCA
End of 848 goes beyond 847.0 by 1
AQENVDYVILKH
AQENVDYVILKHA
End of 804 goes beyond 802.0 by 2
IDAFSDYANFK
IDAFSDYANFKAA
End of 453 goes beyond 451.0 by 2
LSSFTSYENPT
LSSFTSYENPTAA
End of 224 goes beyond 223.0 by 1
EKQFQPYFIPIN
EKQFQPYFIPINA
End of 811 goes beyond 806.0 by 5
DNDDDLYG
DNDDDLYGAAAAA
End of 717 goes beyond 715.0 by 2
GVGGTTYEISV
GVGGTTYEISVAA
Beginning of 0 missing 1 amino acids
MADPKYADLPGI
AM

In [22]:
# Fix final issue
conforming.loc[conforming['Interactor'] == 'MAESITYAAVAR', 'Interactor'] = 'MAESITYAAVARA'

In [23]:
# Ensure all of our "conforming" peptides have the proper length
conforming[conforming['Interactor'].str.len() != 13]

,Spot Index,Spot Flag,Interactor,Interactor Protein,Measure Flag,PTP_HD-PTP_11-07-2008.seam,PTP_LAR_12-10-2007.seam,PTP_LyP_12-10-2007.seam,PTP_MEG-1_12-10-2007.seam,PTP_MEG-2_12-10-2007.seam,...,PTP_TC-PTP_12-10-2007.seam,PTP_PTP1B_averaged,Length,Uniprot_ID,Gene_Name,Peptide_Location,Peptide_Location_in_Fullseq,Fullseq_Length,Tyr_Location,End_of_Peptide


In [24]:
# Now tackle the nonconforming peptide
nonconforming

,Spot Index,Spot Flag,Interactor,Interactor Protein,Measure Flag,PTP_HD-PTP_11-07-2008.seam,PTP_LAR_12-10-2007.seam,PTP_LyP_12-10-2007.seam,PTP_MEG-1_12-10-2007.seam,PTP_MEG-2_12-10-2007.seam,...,PTP_SHP-2_12-10-2007.seam,PTP_DEP-1_12-10-2007.seam.txt,PTP_TC-PTP_12-10-2007.seam,PTP_PTP1B_averaged,Length,Uniprot_ID,Gene_Name,Peptide_Location,Peptide_Location_in_Fullseq,Fullseq_Length
970,5185,GOOD,MSSLYYANALFS,P09629 (HOXB7) --> 1-12,GOOD,3.85,2.51,1.12,6.67,0.199,...,1.18,0.278,5.3,5.74,12,P09629,HOXB7,1-12,Peptide_Not_Found,None_Applicable


In [25]:
# Seeing as this is a short peptide, and the second half of the sequence has 6 AAs
#  which aren't Y, we can assume one A is required to pad the sequence from the front
nonconforming.loc[nonconforming['Interactor'] == 'MSSLYYANALFS', 'Interactor'] = 'AMSSLYYANALFS'

In [26]:
# Now alter our 'peptides' array with the amended sequences
fixed_peps = conforming['Interactor']
fixed_peps = pd.concat([fixed_peps, nonconforming['Interactor']])

peptides.loc[fixed_peps.index, 'Interactor'] = fixed_peps

In [27]:
## FINALLY WE CAN GO ON TO GENERATING OUR FEATURES ##

In [28]:
# METHOD FOR GENERATING FEATURES USING ONE SEQUENCE AT A TIME
def FeatureGen (sequence, protdcal):
    # FIRST: GENERATE PROTDCAL VALUES
    slist = list(sequence)   # first split sequence up into list
    # Go through sequence to get protdcal value
    pd = []
    for i in slist:
        pd.append(protdcal.loc[i].tolist())
    values = list(map(lambda *x: sum(x), *pd))   # add up values
    headers =  protdcal.columns.tolist()   # include headers


    # SECOND: GENERATE ONE-HOT ENCODING
    aa = ['K', 'R', 'H', 'A', 'I', 'L', 'M', 'V', 'F', 'W', 'Y', 'N', 'C', 'Q', 'S', 'T', 'D', 'E', 'G', 'P']   # possible amino acids
    # Make headers and one-hot encoding for each letter
    for i in aa:
        j = 0
        while j < len(sequence):
            headers.append('ONE-HOT_' + str(j) + '-' + i)  # make header
            if sequence[j] == i:
                values.append(1)
            else:
                values.append(0)
            j+=1


    # THIRD: GENERATE MACCS KEYS
    # Generate maccs keys
    mol = (rdmolfiles.MolFromFASTA(sequence))
    fp = (MACCSkeys.GenMACCSKeys(mol))
    maccs = fp.ToBitString()
    binary = list(maccs)   # split up into list
    values.extend(binary)   # add list onto resulting values
    # Generate headers for maccs keys
    mt = list(itertools.chain(range(len(binary))))
    mt = [str(s) + '_maccs' for s in mt]
    headers.extend(mt)   # append header values

    return values, headers

In [29]:
peptides = peptides.rename(columns={'Peptide_Location_in_Fullseq':'SITE_LOC', 'Uniprot_ID':'ACC_ID', 'Interactor':'Peptide'})

In [30]:
# GENERATE FEAUTRES PEPTIDE BY PEPTIDE
# Remove duplicates
feat_dup_len = (len(peptides))
to_drop = peptides[peptides.duplicated(subset='Peptide', keep=False)]
peptides = peptides.drop_duplicates(subset='Peptide')
feat_dup_len = feat_dup_len - (len(peptides))

print("Removed " + str(feat_dup_len) + " duplicate peptide sequences")

Removed 145 duplicate peptide sequences


In [31]:
peptides

,Spot Index,Spot Flag,Peptide,Interactor Protein,Measure Flag,PTP_HD-PTP_11-07-2008.seam,PTP_LAR_12-10-2007.seam,PTP_LyP_12-10-2007.seam,PTP_MEG-1_12-10-2007.seam,PTP_MEG-2_12-10-2007.seam,...,PTP_SHP-2_12-10-2007.seam,PTP_DEP-1_12-10-2007.seam.txt,PTP_TC-PTP_12-10-2007.seam,PTP_PTP1B_averaged,Length,ACC_ID,Gene_Name,Peptide_Location,SITE_LOC,Fullseq_Length
0,352,GOOD,PLSDVLYGRVADF,Q10588 (BST1) --> 128-140,GOOD,0.0134,2.6400,1.110,1.080,10.300,...,3.2500,9.500,3.220,3.6500,13,Q10588,BST1,128-140,127,318.0
1,665,GOOD,NLLGELYGKAGLN,P46019 (PHKA2) --> 805-817,GOOD,-0.4470,3.9000,1.080,1.560,8.520,...,1.6800,9.470,3.190,3.5350,13,P46019,PHKA2,805-817,804,1235.0
2,3368,GOOD,ESLESLYFTPIPA,Q14980 (NUMA1) --> 1768-1780,GOOD,0.9550,1.2100,1.960,2.280,7.790,...,1.8200,8.690,2.970,3.2950,13,Q14980,NUMA1,1768-1780,1767,2115.0
3,6152,BAD_CYS,SKCCCIYEKPRAF,O60927 (PPP1R11) --> 58-70,GOOD,18.2000,21.4000,21.300,6.550,10.400,...,9.1400,8.580,4.260,4.0400,13,O60927,PPP1R11,58-70,57,126.0
4,1177,GOOD,LSGFELYGTVNGV,Q9ULT8 (HECTD1) --> 1233-1245,GOOD,0.4470,1.6700,0.522,0.978,8.060,...,0.7430,8.420,2.330,1.7150,13,Q9ULT8,HECTD1,1233-1245,1230,2610.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,6098,BAD_PTYR,IAEELGYDLLGQI,P29120 (PCSK1) --> 49-61,GOOD,-1.2400,-0.7270,-2.780,-0.954,-0.989,...,-0.6640,-0.347,-0.217,0.1499,13,P29120,PCSK1,49-61,48,753.0
6198,913,GOOD,FSSGHIYVLMGLL,Q99777 --> 9-21,GOOD,-0.9950,-0.0155,-3.590,-0.414,0.290,...,-0.9010,-0.552,0.260,-0.1650,13,Q99777,None_Listed,9-21,8,133.0
6199,5277,BAD_PTYR,EMVESGYVCEGDH,Q9H4A3 (WNK1) --> 539-551,GOOD,-0.6740,-0.6800,0.111,-0.297,-1.550,...,-0.0844,-0.638,-0.236,-0.6555,13,Q9H4A3,WNK1,539-551,538,2382.0
6200,6099,BAD_PTYR,NNDEKFYLSNGRI,Q9UPC6 (hCLCA1) --> 170-182,GOOD,0.2200,0.1700,0.134,0.240,-0.261,...,-0.2360,-1.050,-0.196,-0.3065,13,Q9UPC6,hCLCA1,170-182,169,914.0


In [32]:
# Format user defined features for FeatureGen call
temp_annotation = peptides['SITE_LOC'].astype(str)
peptides['uid_pos'] = peptides['ACC_ID'] + "_" + temp_annotation
peptides['uid_pos'] = peptides['uid_pos'] + '_c' + peptides.groupby('uid_pos').cumcount().astype(str) ## ADDED IN TO ADDRESS SAME UID_POS BUT DIFF SEQ ISSUE
peptides = peptides.set_index('uid_pos', drop=True)

In [33]:
# Pull sequences and make X -> A for feature generation (A is least offensive AA)
sequences = peptides['Peptide'].str.replace('X', 'A')

In [34]:
## NOTE: REMOVED THE FOLLOWING FOR THE 2004 DATASET AS THERE WERE NO UNIPROT IDs!
## IF USING THE CESARENI DATASET THIS LIKELY SHOULD BE CHANGED BACK !!

# ** WILL NEED TO IMPORT THE GO TABLE !!! **

# Pull user defined features' uniprot IDs from GO table - generate associated GO annotations
#feat_uids = pd.DataFrame(peptides['ACC_ID'])
#feat_uids = feat_uids.rename(columns={'ACC_ID':'uid'})
#feat_uids = feat_uids.drop_duplicates().reset_index(drop=True)
#goa_uids = pd.merge(feat_uids, goa, on='uid')
#goa_counts = pd.DataFrame(goa_uids['go_term'].value_counts(), columns=['count'])
#goa_counts = goa_counts.reset_index(drop=False).rename(columns={'index':'go_term', 'count':'go_count'})
#goa_unique = goa_uids.drop_duplicates(subset='go_term')
#mapped_goa = pd.merge(goa_counts, goa_unique, on='go_term').drop(columns=['uid', 'gene', 'go_association', 'go_ref', 'go_b_p_f'])   # finalised df with counts to be displayed to user

In [35]:
# Now import our testing peptides (from Arrington et. al., 2019)
#testing = pd.read_csv('../arrington_2019_MS_Verified_Substrates.csv')

In [36]:
# Format testing data for feature generation
#testing = testing.set_index(testing['UniProt Accession'] + '_' + testing['Site'])

In [37]:
#testing

In [38]:
# Replace gaps with least reactive AA (Alanine)
#testing_cleaned = testing['Sequence'].str.replace('_', 'A')

In [39]:
# Be sure no testing peptides exist within the training (Rychlewski) peptides dataset
#repeated = []
#for t in testing_cleaned:
#    if sequences.str.contains(t).any():
#        repeated.append(t)

#print('Number of testing peptides found within our training peptides: ' + str(len(repeated)))

In [40]:
# Ensure no repeats within our testing peptides
#testing_cleaned = testing_cleaned.drop_duplicates()

In [41]:
## GENERATE FEATURES FOR OUR TRAINING DATA
# Create df for results to go into
v, h = FeatureGen(sequences[0], protdcal)
features = pd.DataFrame(columns=h)
features.loc[len(features)] = v

i = 1

# Go through rest of sequences to generate feature set
while i < len(sequences):
    ts = sequences[i]
    value, header = FeatureGen(ts, protdcal)
    features.loc[len(features)] = value
    i+=1
    if i % 500 == 0:
        print(i, 'of', len(sequences), 'completed')

# Make the index the same as our initial dataframe
feat_x = features.set_index(peptides.index)

# Isolate the methylated condition from the sequences as our y value
#feat_y = peptides['Experimental_Phosphorylation']

500 of 6057 completed
1000 of 6057 completed
1500 of 6057 completed
2000 of 6057 completed
2500 of 6057 completed
3000 of 6057 completed
3500 of 6057 completed
4000 of 6057 completed
4500 of 6057 completed
5000 of 6057 completed
5500 of 6057 completed
6000 of 6057 completed


In [42]:
feat_x.to_csv('./features/palma_2017_x_features.csv')
#feat_y.to_csv('./features/palma_2017_y_features.csv')
peptides.to_csv('./features/palma_2017_feature_details.csv')

In [43]:
## GENERATE FEATURES FOR OUR TESTING DATA
# Create df for results to go into
#v_test, h_test = FeatureGen(testing_cleaned[0], protdcal)
#features_testing = pd.DataFrame(columns=h_test)
#features_testing.loc[len(features_testing)] = v_test

#i = 1

# Go through rest of sequences to generate feature set
#while i < len(testing_cleaned):
#    ts = testing_cleaned[i]
#    value, header = FeatureGen(ts, protdcal)
#    features_testing.loc[len(features_testing)] = value
#    i+=1
#    if i % 500 == 0:
#        print(i, 'of', len(testing_cleaned), 'completed')

# Make the index the same as our initial dataframe
#feat_x_test = features_testing.set_index(testing.index)

# All of these are positive for PTM activity so no y-set required

In [44]:
#feat_x_test.to_csv('./known_Abl_substrates/arrington_2019_finaltest_x_features.csv')
#testing.to_csv('./known_Abl_substrates/arrington_2019_finaltest_feature_details.csv')

In [45]:
# As we've already somewhat handled our full protein sequences, we should format them
#  into a .fasta file for the MuSite Deep run (the results from this are imported
#  and cleaned in another notebook)
f = open('./palma_2017_fullseqs_for_MSD.fasta', 'w')

for h,r in fullseqs.iterrows():
    f.write('>' + r['Entry Name'] + '|' + r['From'] + '\n')
    f.write(str(r['Sequence']) + '\n')
f.close()